In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
import re
import numpy as np
import base64
import numpy as np
import cv2
import undetected_chromedriver as uc
import argparse
from urllib.parse import urlparse
import io
from PIL import Image
import time
import os
import base64
import numpy as np
import cv2
import undetected_chromedriver as uc
import argparse
from urllib.parse import urlparse
import io
from PIL import Image
import time
import os

In [2]:
# Funciones

def download_image_to_numpy(driver ,url, wait_time=10):
    """
    Downloads an image from a URL that blocks normal requests (like Ripley's CDN)
    and returns it as a numpy array using undetected-chromedriver to bypass protection.
    
    Args:
        url: URL of the image to download
        wait_time: Time to wait for Cloudflare challenges to resolve (seconds)
        
    Returns:
        numpy array of the image
    """

    
    try:

        # Now navigate to the actual image URL
        print(f"Now navigating to the image URL...")
        driver.get(url)
        
        # Additional wait for any challenges
        print(f"Waiting {wait_time} seconds for any Cloudflare challenges...")
        time.sleep(wait_time)
        
        # Check if we're on a Cloudflare challenge page
        page_source = driver.page_source
        if "Just a moment..." in page_source or "Checking your browser" in page_source:
            print("Detected Cloudflare challenge page, waiting for it to resolve...")
            # Wait longer for the challenge to resolve
            time.sleep(wait_time * 2)
        
        # If it's a direct image, the content should be binary, not HTML
        content_type = driver.execute_script("return document.contentType")
        print(f"Content type: {content_type}")
        
        if content_type and "image" in content_type.lower():
            print("Direct image detected, downloading...")
            # For direct image URLs that load in the browser
            get_base64 = """
            const callback = arguments[0];
            let canvas = document.createElement('canvas');
            let img = document.querySelector('img');
            if (!img) {
                // If no img tag (direct image), convert the body to base64
                let reader = new FileReader();
                reader.onloadend = function() {
                    callback(reader.result.split(',')[1]);
                }
                fetch(window.location.href)
                    .then(response => response.blob())
                    .then(blob => reader.readAsDataURL(blob));
            } else {
                // If there's an img tag
                canvas.width = img.naturalWidth;
                canvas.height = img.naturalHeight;
                canvas.getContext('2d').drawImage(img, 0, 0);
                callback(canvas.toDataURL('image/png').split(',')[1]);
            }
            """
        else:
            # For URLs that require fetch
            print("Using fetch to get image content...")
            get_base64 = """
            const url = arguments[0];
            const callback = arguments[1];
            fetch(url, { 
                credentials: 'include',
                headers: {
                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
                }
            })
            .then(resp => {
                if (!resp.ok) {
                    throw new Error('Network response was not ok: ' + resp.status);
                }
                return resp.blob();
            })
            .then(blob => {
                if (blob.type.startsWith('text/html')) {
                    throw new Error('Received HTML instead of an image');
                }
                const reader = new FileReader();
                reader.onloadend = () => callback(reader.result.split(',')[1]);
                reader.readAsDataURL(blob);
            })
            .catch(error => {
                console.error('Error:', error);
                callback(null);
            });
            """
        
        # Execute the script and get the base64 image data
        base64_str = driver.execute_async_script(get_base64, url) if "image" not in content_type.lower() else driver.execute_async_script(get_base64)
        
        if not base64_str:
            # If we couldn't get the image, take a screenshot as fallback
            print("Failed to get image through JavaScript, trying screenshot of the page...")
            if "image" in content_type.lower():
                screenshot = driver.get_screenshot_as_base64()
                if screenshot:
                    base64_str = screenshot
                    print("Retrieved image via screenshot")
                else:
                    raise RuntimeError("Failed to take screenshot")
            else:
                raise RuntimeError("Failed to download image. Received non-image content.")
        
        # Decode the Base64 to bytes
        img_bytes = base64.b64decode(base64_str)
        
        # Check if we got HTML instead of an image
        if img_bytes[:20].find(b'<!DOCTYPE') != -1 or img_bytes[:20].find(b'<html') != -1:
            # Save the HTML for debugging
            with open("error_page.html", "wb") as f:
                f.write(img_bytes)
            raise RuntimeError("Received HTML instead of an image. Saved to error_page.html for debugging.")
        
        # Intenta primero con OpenCV
        buf = np.frombuffer(img_bytes, dtype=np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_UNCHANGED)
        
        # Si OpenCV falla, intenta con PIL/Pillow
        if img is None:
            print("OpenCV failed to decode the image, trying with PIL...")
            pil_img = Image.open(io.BytesIO(img_bytes))
            img = np.array(pil_img)
            
            # Convertir RGBA a BGR si es necesario para OpenCV
            if len(img.shape) == 3 and img.shape[2] == 4:
                # Convertir RGBA a RGB
                rgb_img = pil_img.convert('RGB')
                img = np.array(rgb_img)
                # Convertir RGB a BGR para OpenCV
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            elif len(img.shape) == 3 and img.shape[2] == 3:
                # Convertir RGB a BGR para OpenCV
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                
        if img is None:
            raise RuntimeError("Failed to decode the image with both OpenCV and PIL.")
            
        return img
        
    except Exception as e:
        raise RuntimeError(f"Error downloading image: {str(e)}")
    
    #finally:
        # Always quit the driver to free resources
    #    driver.quit()

In [3]:
import undetected_chromedriver as uc


driver = uc.Chrome(headless=False)  

driver.get("https://simple.ripley.com.pe/moda/especiales/ver-todo-polos?source=search&term=polos&s=mdco")

driver.maximize_window()



In [4]:
def scrap_ripley(driver):
    
    

    descripciones = []
    precios_regular = []
    precios_oferta = []
    imagenes = []
    for i in range(1,50):
        try:
            descripcion = driver.find_element(By.XPATH, f'/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[3]').text ## Descripcion
            descripciones.append(descripcion)
            
        except:
            descripciones.append("")
        try:
            precio_regular = driver.find_element(By.XPATH, f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[4]/div/ul/li[1]").text # Precio regular
            precios_regular.append(precio_regular)
        except:
            precios_regular.append(" ")
            
            
        try:
            precio_oferta = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[4]/div/ul/li[2]").text # Precio oferta
            precios_oferta.append(precio_oferta)
        except:
            precios_oferta.append(" ")
            
            
            
        try:
            imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[1]/img").get_attribute("src") # Imagen
            imagenes.append(imagen)
                
        except:
            try:
                imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[4]/img").get_attribute("src") # Imagen                                           
                imagenes.append(imagen)
            except:
                try:
                    imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[3]/img").get_attribute("src") # Imagen
                    imagenes.append(imagen)
                except:
                    try:
                        imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[2]/img").get_attribute("src") # Imagen
                        imagenes.append(imagen)
                            
                    except:
                        imagenes.append("")
        
        
        inicio = 0
        fin = 500
        ActionChains(driver)\
        .scroll_by_amount(inicio, fin)\
        .perform()
        inicio = fin
        fin = fin + 1000

  
    return descripciones, precios_regular, precios_oferta, imagenes



In [5]:

descripciones, precios_regular, precios_oferta, imagenes = scrap_ripley(driver)

In [6]:
final_descripciones = []
final_precios_regular = []
final_precios_oferta = []
final_imagenes = []

final_descripciones.extend(descripciones)
final_precios_regular.extend(precios_regular)
final_precios_oferta.extend(precios_oferta)
final_imagenes.extend(imagenes)

In [16]:
page_number = 5
driver.get(f"https://simple.ripley.com.pe/moda/especiales/ver-todo-polos?source=search&term=polos&s=mdco&page={page_number}")
driver.implicitly_wait(5)
descripciones, precios_regular, precios_oferta, imagenes = scrap_ripley(driver)
final_descripciones.extend(descripciones)
final_precios_regular.extend(precios_regular)
final_precios_oferta.extend(precios_oferta)
final_imagenes.extend(imagenes)

In [17]:
print(len(final_precios_oferta))
print(len(final_precios_regular))
print(len(final_descripciones))


final_descripciones


245
245
245


['POLO MANGA LARGA MUJER INDEX FANAL PANAL',
 'POLO MANGA LARGA MUJER INDEX AMADA',
 'POLO MUJER INDEX EVUELO',
 'POLO MUJER NAVIGATA LOGAN-F R REGATTA CUELLO MOCK DISENO',
 'POLO MANGA CORTA MUJER INDEX ESIMO',
 'POLO MUJER INDEX FASIMETRIC',
 '',
 'POLO MANGA CORTA HOMBRE KENNETH STEVENS PIQUÉ',
 'POLO MANGA LARGA HOMBRE KENNETH STEVENS BURITI',
 'POLO MANGA CORTA HOMBRE KENNETH STEVENS PIQUÉ',
 'POLO MUJER INDEX EJAZ',
 'POLO MUJER CACHAREL ARCO',
 'POLO MANGA LARGA ALGODÓN MUJER INDEX CUADRO',
 'POLO MUJER CACHAREL FARI',
 'POLO MUJER SPORTY TANK ROJO LEVIS A7420-0007',
 'POLO MUJER REGULAR FIT LILA LEVIS 17369-2746',
 'POLO MANGA CORTA HOMBRE INDEX TRENDY',
 'POLO MANGA CORTA HOMBRE KENNETH STEVENS BLANCO PIQUÉ',
 'POLO MANGA CORTA HOMBRE INDEX TRENDY',
 'POLO MANGA CORTA ALGODÓN HOMBRE US POLO USLPM-627PER',
 'POLERA HOMBRE HALF HOODIE DOHA-M1',
 'POLERA HOMBRE HALF HOODIE BREWY-M1',
 'POLO HOMBRE NAUTICA AZUL K41050-4NV',
 'POLO MANGA CORTA ALGODÓN HOMBRE LACOSTE PH4012 CAMISERO

In [19]:
data = pd.DataFrame({"descripcion": final_descripciones, "precio_regular": final_precios_regular, "precio_oferta": final_precios_oferta, "imagen": final_imagenes})

data

,descripcion,precio_regular,precio_oferta,imagen
0,POLO MANGA LARGA MUJER INDEX FANAL PANAL,S/ 69.99,S/ 41.99,https://rimage.ripley.com.pe/home.ripley/Attac...
1,POLO MANGA LARGA MUJER INDEX AMADA,S/ 59.99,S/ 35.99,https://rimage.ripley.com.pe/home.ripley/Attac...
2,POLO MUJER INDEX EVUELO,S/ 69.99,S/ 41.99,https://rimage.ripley.com.pe/home.ripley/Attac...
3,POLO MUJER NAVIGATA LOGAN-F R REGATTA CUELLO M...,S/ 99.99,S/ 59.99,https://rimage.ripley.com.pe/home.ripley/Attac...
4,POLO MANGA CORTA MUJER INDEX ESIMO,S/ 89.99,S/ 53.99,https://rimage.ripley.com.pe/home.ripley/Attac...
...,...,...,...,...
240,POLO MANGA CORTA ALGODÓN HOMBRE JOHN HOLDEN LI...,S/ 99.90,S/ 79.90,https://rimage.ripley.com.pe/home.ripley/Attac...
241,POLO MANGA CORTA ALGODÓN HOMBRE PIERRE CARDIN ...,S/ 229,,https://rimage.ripley.com.pe/home.ripley/Attac...
242,POLO MANGA CORTA ALGODÓN ORGÁNICO HOMBRE SELEC...,S/ 129.99,S/ 99.99,https://rimage.ripley.com.pe/home.ripley/Attac...
243,POLO MANGA CORTA HOMBRE SELECTED SLHFIGO ZIP,S/ 149.99,S/ 99.99,https://rimage.ripley.com.pe/home.ripley/Attac...
